## Quantifying results from Google query and using logistic regression to predict likelyhood of VIP

Will need to create a training dataset. Could integrate ChatGPT using snippets or websites from the following promt: "Using the following snippets. Give a rating from 0-100 on the likeliness that the person is famous. Respond only with the rating. snippets: "

In [103]:
#import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os
import json
import instaloader
import requests

#load evn variables
API_KEY = os.environ['GOOGLE_API_KEY']
SEARCH_ENGINE_ID = os.environ['SEARCH_ENGINE_ID']

# Functions to Quantify Google query results

In [116]:
#define functions


#google name and output json file
#takes in name api_key, search_engine_id
def search_name(name, api_key, search_engine_id):   
    # Iterate over the names and grab search results using Google Custom Search API
    search_url = f'https://www.googleapis.com/customsearch/v1?q={name}&cx={search_engine_id}&key={api_key}'
    response = requests.get(search_url)
    data = json.loads(response.text)
    return data


# checks email domain for company emails, 
# input takes in dataframe object
# returns dataframe with 1 or 0 for check email column 
def check_email(df):
    xlst = ['graduatehotels.com','schultehospitality.com','schultedc.com']
    lst = []
    for i in range(len(df)):
        if df.loc[i] == "nan":
            lst.append(0)
        else:
            if df.loc[i].split("@")[1].lower() in xlst:
                lst.append(1)
            else:
                lst.append(0)
    return lst


#clean data set to add full names and split email for domain, 
# takes in string file path
# return dataframe object
def clean_data(file_path):
    df = pd.read_excel(file_path)
    df['names'] = df['First Name'] + " " + df['Last Name']
    df['Email'].fillna("nan",inplace=True)
    lenOfDF = [0 for i in range(len(df))]
    emails = df['Email']
    df['Email Check'] = check_email(emails)
    df['Websites'] = lenOfDF
    df['Social Media'] = lenOfDF
    df['keywords'] = lenOfDF
    return df


# instagram follower count
# takes in url of account from json['items'][int]['metatags']['og:url']
#  returns int follower count,
# WILL NEED TWEEPY FOR TWITTER FOLLOWER COUNT, DO NOT HAVE ACCESS YET
def instaCount(link):
    bot = instaloader.Instaloader()
    profile = instaloader.Profile.from_username(bot.context,link[26:-1])
    return profile.followers


# scans json for websites associated with name 
# takes in jsonObj and scans through for specific links
# links will be weighted using ints will need to tune weights later
# returns wighted score 
def scan_websites(jsonObj,firstName,lastName):
    WEBSITES = {'en.wikipedia.org':10} # add the rest of links
    score = 0
    k = 10 #only checks first 10, change for more, can do len for full
    first = firstName.lower()
    last = lastName.lower()
    if 'items' in jsonObj:
        for j in range(0,k): #only checks first 10
            if jsonObj['items'][j]['displayLink'] in WEBSITES:
                score += WEBSITES[jsonObj['items'][j]['displayLink']]
            if first in jsonObj['items'][j]['displayLink'].lower() or last in jsonObj['items'][j]['displayLink'].lower():
                score += 15 # adjust for weight
    return score


#scans for keywords in snippet 
#will take in jsonObj and look for key words in the snippet
#could also weigh words 
#can also combine with scan websites 
#outputs int score
def scan_keywords(jsonObj):
    KEYWORDSW = {'owner':10} #add more words
    KEYWORDS = ['onwer'] #add more words
    k = 10 #only checks first 10, change for more, can do len for full
    score = 0
    if 'items' in jsonObj:
        for j in range(0,k): #only checks first 10
            for x in jsonObj['items'][j]['snippet'].split(" "):
                if x.lower() in KEYWORDS:
                    score +=1
    return score

#scans json for websites and keywords. combines the 2 functions above
def scan_combined(jsonObj):
    WEBSITES = {'en.wikipedia.org':10} # add the rest of links
    KEYWORDS = ['owner'] #{owner:10} #uncomment for dictionary
    score = 0
    k = 10 #only checks first 10, change for more, can do len for full
    name = jsonObj[0]['queries']['request'][0]['searchTerms'].split(" ")
    if 'items' in jsonObj:
        for j in range(0,k): #only checks first 10
            if jsonObj['items'][j]['displayLink'] in WEBSITES:
                score += WEBSITES[jsonObj['items'][j]['displayLink']]
            if name[0] in jsonObj['items'][j]['displayLink'] or name[1] in jsonObj['items'][j]['displayLink']:
                score += 15 # adjust for weight
            for x in jsonObj['items'][j]['snippet'].split(" "):
                if x.lower() in KEYWORDS:
                    score += 10
                    #score += KEYWORDS[x.lower()] #uncomment for dic
    return score


## Test function

In [112]:
test = pd.read_excel("test file.xlsx")
test.head(1)

,Arrival Date,Last Name,First Name,Email,Address,City
0,2023-05-08,Smith,Carolyn,csura@lwdirect.com,NaN,NaN


In [117]:
test = clean_data('test file.xlsx')
test

,Arrival Date,Last Name,First Name,Email,Address,City,names,Email Check,Websites,Social Media,keywords
0,2023-05-08,Smith,Carolyn,csura@lwdirect.com,NaN,NaN,Carolyn Smith,0,0,0,0
1,2023-05-08,Williams,Scotty,macy@namstatepageant.com,NaN,NaN,Scotty Williams,0,0,0,0
2,2023-05-08,Williams,Mayson,macy@namstatepageant.com,NaN,NaN,Mayson Williams,0,0,0,0
3,2023-05-08,Williams,Rebecca,macy@namstatepageant.com,NaN,NaN,Rebecca Williams,0,0,0,0
4,2023-05-08,Chung,Sunku,v9outiu5q5@m.expediapartnercentral.com,NaN,NaN,Sunku Chung,0,0,0,0
5,2023-05-08,Fowler,Thomas,fompvyinai@m.expediapartnercentral.com,NaN,NaN,Thomas Fowler,0,0,0,0
6,2023-05-08,bernstein,Mark,MarkBernstein@icloud.com,1025 Memorial Village Dr,Houston,Mark bernstein,0,0,0,0
7,2023-05-08,WREN,PATRICK R,nan,NaN,NaN,PATRICK R WREN,0,0,0,0
8,2023-05-09,Virtue,Ed,evirtue@graduatehotels.com,NaN,NaN,Ed Virtue,1,0,0,0
9,2023-05-10,Cuban,Mark,mcuban@gmail.com,NaN,NaN,Mark Cuban,0,0,0,0


In [67]:
jsondata = []
for i in range(len(test)):
    jsondata.append(str(search_name(test['names'].loc[i],API_KEY,SEARCH_ENGINE_ID)))
jsondata

[{'kind': 'customsearch#search',
  'url': {'type': 'application/json',
   'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&relatedSite={relatedSite?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
  'queries': {'request': [{'title': 'Google Custom Search - Carolyn Smith',
     'totalResults': '34900000',
     'searchTerms': 'Carolyn Smith',
     'count': 10,
     'startIndex': 1,
     'inputEncoding': 'utf8',
     'outp

In [68]:
test['data'] = jsondata
test.head()

,Arrival Date,Last Name,First Name,Email,Address,City,names,Email Check,Websites,Social Media,keywords,data
0,2023-05-08,Smith,Carolyn,csura@lwdirect.com,NaN,NaN,Carolyn Smith,0,0,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
1,2023-05-08,Williams,Scotty,macy@namstatepageant.com,NaN,NaN,Scotty Williams,0,0,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
2,2023-05-08,Williams,Mayson,macy@namstatepageant.com,NaN,NaN,Mayson Williams,0,0,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
3,2023-05-08,Williams,Rebecca,macy@namstatepageant.com,NaN,NaN,Rebecca Williams,0,0,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
4,2023-05-08,Chung,Sunku,v9outiu5q5@m.expediapartnercentral.com,NaN,NaN,Sunku Chung,0,0,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."


In [73]:
print(instaCount('https://www.instagram.com/mcuban/'))

1970777


In [96]:
for i in range(len(test)):
    test['Websites'].loc[i] = scan_websites(test['data'].loc[i],test['First Name'].loc[i],test['Last Name'].loc[i])
test.head()

C:\Users\GakuTokusumi\AppData\Local\Temp\ipykernel_15104\2609113178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Websites'].loc[i] = scan_websites(test['data'].loc[i],test['First Name'].loc[i],test['Last Name'].loc[i])
C:\Users\GakuTokusumi\AppData\Local\Temp\ipykernel_15104\2609113178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Websites'].loc[i] = scan_websites(test['data'].loc[i],test['First Name'].loc[i],test['Last Name'].loc[i])
C:\Users\GakuTokusumi\AppData\Local\Temp\ipykernel_15104\2609113178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,Arrival Date,Last Name,First Name,Email,Address,City,names,Email Check,Websites,Social Media,keywords,data
0,2023-05-08,Smith,Carolyn,csura@lwdirect.com,NaN,NaN,Carolyn Smith,0,0,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
1,2023-05-08,Williams,Scotty,macy@namstatepageant.com,NaN,NaN,Scotty Williams,0,25,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
2,2023-05-08,Williams,Mayson,macy@namstatepageant.com,NaN,NaN,Mayson Williams,0,70,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
3,2023-05-08,Williams,Rebecca,macy@namstatepageant.com,NaN,NaN,Rebecca Williams,0,15,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
4,2023-05-08,Chung,Sunku,v9outiu5q5@m.expediapartnercentral.com,NaN,NaN,Sunku Chung,0,15,0,0,"{'kind': 'customsearch#search', 'url': {'type'..."
